In [2]:
import pandas as pd

file_name = '/content/drive/MyDrive/Dataset/fashion-mnist_train.csv'
df = pd.read_csv(file_name)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999 entries, 0 to 6998
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 41.9 MB


In [ ]:
# Split the dataset into training and test dataset

X = df.iloc[:,1:].values
y = df.iloc[:,0].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=24
)

In [ ]:
# Feature Scaling

X_train = X_train / 255.0
X_test  = X_test  / 255.0

In [ ]:
# Create Custom Dataset

import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)  # long for CrossEntropy
    def __len__(self):      return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

In [ ]:
# Create Train Dataset

train_dataset = CustomDataset(X_train, y_train)

In [ ]:
# Create Test Dataset

test_dataset = CustomDataset(X_test, y_test)


In [ ]:
# Create Train & Test dataset loader

from torch.utils.data import DataLoader

train_loader = DataLoader(CustomDataset(X_train, y_train),
                          batch_size=32, shuffle=True)
test_loader  = DataLoader(CustomDataset(X_test,  y_test),
                          batch_size=32, shuffle=False)

In [ ]:
# Create NN module

from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt

class NN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64,10)
    )

  def forward(self,X):

    return self.model(X)


In [ ]:
# Set Values of Epochs & Learning Rate

epochs = 1000
Learning_rate = 0.001

In [ ]:
# Instantiation of the model
model= NN(784)

#loss function
Loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.SGD(model.parameters(), lr= Learning_rate)

In [ ]:
# Training Loop

for epoch in range(epochs):

  for batch_features, batch_labels in train_loader:

    #Forward pass:
    outputs = model(batch_features)

    #calculate loss:
    loss=Loss_fn(outputs, batch_labels.long())

    #back pass:
    optimizer.zero_grad()
    loss.backward()

    #update grad:
    optimizer.step()


In [ ]:
#Evaluation code:

correct = 0
total   = 0
model.eval()
with torch.no_grad():
    for Xb, yb in test_loader:
        logits = model(Xb)
        preds  = logits.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total   += yb.size(0)

print(f"Test accuracy: {correct/total*100:.2f}%")

Test accuracy: 84.71%
